In [1]:
import json

def generate_docker_compose_and_config(num_servers):
    base_port = 50051
    docker_compose = {
        'version': '3.8',
        'services': {},
        'networks': {
            'mynetwork': None
        }
    }

    config = {
        'servers': []
    }

    # Generate server configurations
    for i in range(num_servers):
        server_name = f'server-{i}'
        port = base_port + i

        docker_compose['services'][server_name] = {
            'image': 'zookeeper_server:latest',
            'ports': [f"{port}:{port}"],
            'networks': ['mynetwork'],
            'command': 'bash -c "go run *.go"',
            'tty': True,
            'environment': [f'PORT={port}', f'ID={i}']
        }

        if i != 0:  # For servers other than the first one, add a dependency on server-0
            docker_compose['services'][server_name]['depends_on'] = ['server-0']
        
        config['servers'].append({'host': server_name, 'port': port})

    # Generate client configuration
    client_env = [f'SERVER_{i}_ADDR=server-{i}:{base_port + i}' for i in range(num_servers)]
    docker_compose['services']['client'] = {
        'build': {
            'context': '.',
            'dockerfile': './client/Dockerfile'
        },
        'image': 'zookeeper_client:latest',
        'ports': ['50057:50057'],
        'networks': ['mynetwork'],
        'tty': True,
        'environment': client_env
    }

    # Convert docker-compose to YAML format
    docker_compose_yaml = convert_to_yaml(docker_compose)

    return docker_compose_yaml, json.dumps(config, indent=4)


def convert_to_yaml(docker_compose_dict):
    """
    Convert the Docker Compose dictionary to a YAML-formatted string.
    This is a simplified converter for demonstration purposes.
    """
    def convert_item(key, value):
        if isinstance(value, dict):
            return f"{key}:\n" + "\n".join([f"  {k}: {v}" for k, v in value.items() if v is not None])
        elif isinstance(value, list):
            return f"{key}:\n" + "\n".join([f"  - {v}" for v in value])
        else:
            return f"{key}: {value}"

    lines = []
    for key, value in docker_compose_dict.items():
        lines.append(convert_item(key, value))

    return "\n".join(lines)


# Example usage
num_servers = 6  # number of servers
docker_compose_yaml, config_json = generate_docker_compose_and_config(num_servers)

# Output the generated files
docker_compose_yaml, config_json



('version: 3.8\nservices:\n  server-0: {\'image\': \'zookeeper_server:latest\', \'ports\': [\'50051:50051\'], \'networks\': [\'mynetwork\'], \'command\': \'bash -c "go run *.go"\', \'tty\': True, \'environment\': [\'PORT=50051\', \'ID=0\']}\n  server-1: {\'image\': \'zookeeper_server:latest\', \'ports\': [\'50052:50052\'], \'networks\': [\'mynetwork\'], \'command\': \'bash -c "go run *.go"\', \'tty\': True, \'environment\': [\'PORT=50052\', \'ID=1\'], \'depends_on\': [\'server-0\']}\n  server-2: {\'image\': \'zookeeper_server:latest\', \'ports\': [\'50053:50053\'], \'networks\': [\'mynetwork\'], \'command\': \'bash -c "go run *.go"\', \'tty\': True, \'environment\': [\'PORT=50053\', \'ID=2\'], \'depends_on\': [\'server-0\']}\n  server-3: {\'image\': \'zookeeper_server:latest\', \'ports\': [\'50054:50054\'], \'networks\': [\'mynetwork\'], \'command\': \'bash -c "go run *.go"\', \'tty\': True, \'environment\': [\'PORT=50054\', \'ID=3\'], \'depends_on\': [\'server-0\']}\n  server-4: {\'im

In [ ]:
def save_to_file(filename, content):
    with open(filename, 'w') as file:
        file.write(content)

# Saving the generated Docker Compose YAML and Config JSON to files
docker_compose_filename = './docker-compose.yml'
config_json_filename = './server/config.json'

docker_compose_yaml, config_json = generate_docker_compose_and_config(6)

save_to_file(docker_compose_filename, docker_compose_yaml)
save_to_file(config_json_filename, config_json)

docker_compose_filename, config_json_filename

